# Thesis work--1

### Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import randrange, uniform

### Global variables and functions for tuning

In [2]:
FEATURES = 4
ROWS = 10
LOW_RANGE = 0
HIGH_RANGE = 10
SELECTION_X_GENERATOR = ['random_normal_dist','random_normal_range','set_manual_x']
SELECTED_GENERATOR = SELECTION_X_GENERATOR[0]
APPLY_FEATURE_SCALING = False
formula = lambda X: X[:,2] - X[:,0] + 5*X[:,1]**2 + (abs(X[:,3])*100)**2

### Generating random numbers using normal distribution

In [3]:
def rnd_norm():
    X = []
    for itr in range(FEATURES):
        X.append(np.random.normal(size=ROWS))
    X = np.asarray(X)
    X = X.reshape(ROWS,FEATURES)
    return X
    

### GENERATE NUMBER USING RANGES 

In [4]:
def rnd_rng():
    X = []
    for feat_ in range(FEATURES):
        temp_ = []
        for rows_ in range(ROWS):
            temp_.append(uniform(LOW_RANGE, HIGH_RANGE))
        X.append(temp_)
        temp_ = []
    X = np.asarray(X)
    X = X.reshape(ROWS,FEATURES)
    return X

### MANUALLY SET X

In [5]:
def mnl_x():
    X = [[4.77483186, 3.39440415, 57.05801446, 6.71629158],
 [6.61347886, 7.2255493,  122.77241787, 7.31532667],
 [4.5746061,  6.94376097, 1.57398063, 7.61596309],
 [3.99442041, 7.74271857, -22.18103276, 8.69864701],
 [7.74195296, 8.66056046, 1000.71418768, 8.66512174],
 [0.70227095, 8.48112974, 12.18087887, 9.98354661],
 [3.30624855, 1.28647579, 29.10406468, 8.96630542],
 [5.7062847,  7.72417331, 56.45891841, 5.15614429],
 [3.70369596, 1.79230683, 157.60607582, 9.4736015 ],
 [5.36628365, 9.68201719, 888.00342486, 2.75472882]]
    X = np.asarray(X)
    return X

### SELECT GENERATION METHOD

In [6]:
if 'random_normal_dist' == SELECTED_GENERATOR:
    X = rnd_norm()
elif 'random_normal_range' == SELECTED_GENERATOR:
    X = rnd_rng()
elif 'set_manual_x' == SELECTED_GENERATOR:
    X = mnl_x()

### Generating random numbers Y

In [7]:
Y = formula(X)
print(np.shape(Y))
print(Y)

(10,)
[11290.28277328   982.86014465 18985.85169269  5034.63953232
  1615.6573557   2977.23164715  1716.53602841 13718.2907906
   967.78255613 30205.53968873]


### Splitting the dataset into the Training set and Test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
if APPLY_FEATURE_SCALING:
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

### Applying PCA

In [18]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1)
#print(X_train)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
print(X_train_pca)
#print(X_test_pca)
# print(np.shape(X_train),np.shape(X_test))
#print(explained_variance)

[[ 0.27225073]
 [ 2.45562474]
 [-2.26129587]
 [-0.83215731]
 [ 0.56763211]
 [-2.39196734]
 [ 1.07633142]
 [ 1.11358152]]


### Applying LDA

In [11]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# lda = LDA(n_components = None)
# X_train_lda = lda.fit_transform(X_train, y_train)
# X_test_lda = lda.transform(X_test)

### Applying Kernel PCA

In [12]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components = 2, kernel = 'rbf')
X_train_pca = kpca.fit_transform(X_train)
X_test_pca = kpca.transform(X_test)

### Applying without difficult vector

In [30]:
# Genrate numpy array having same features given in the state
# Number of rows should be equal to rows in test
# Each column should repeat the same column value
rows_test = len(X_train_pca)
transformed_pca = X_train_pca*np.ones((rows_test,FEATURES),dtype=int)
# predicted Y
predicted_y = formula(transformed_pca)
# Calculating the error between the actual and predicted value
from sklearn.metrics import mean_squared_error
error = mean_squared_error(y_train, predicted_y)
print(error)

797270641.9136169


### Applying with difficult vector
### Calculating DV

In [51]:
# sort X_train_pca and y_train by index
sorted_indexes = np.argsort(X_train_pca,axis=0)
sorted_x_train_pca = X_train_pca[sorted_indexes]
sorted_y_train = y_train[sorted_indexes]

In [64]:
# calculating slope
m_slope = []
m_diff = []
m_diff_trans = []
max_diff = 0
for itr in range(0,len(sorted_x_train_pca)-1):
    m_slope.append((y_train[itr+1] - y_train[itr])/(sorted_x_train_pca[itr+1]-sorted_x_train_pca[itr]))
for itr in range(1,len(sorted_x_train_pca)-1):
    m_diff.append(m_slope[itr]-m_slope[itr-1])
max_diff = max(m_diff)
m_diff_trans = (m_diff/max_diff)
print(np.shape(m_diff_trans),np.shape(sorted_x_train_pca),np.shape(y_train),np.shape(m_slope))
print(m_diff_trans)

(6, 1, 1) (8, 1, 1) (8,) (7, 1, 1)
[[[-1.29314223]]

 [[ 0.11411526]]

 [[ 0.21603053]]

 [[-0.31188993]]

 [[ 1.        ]]

 [[-0.94121302]]]


### Sampling techniques